# <font color = "red">Classificação de Risco de Crédito</font>

## MODELAGEM PREDITIVA (CLASSIFICAÇÃO)

<font color = "red">Problema de Negócio:</font> A partir de dados históricos, irei verificar os padrões que levaram os clientes de uma instituição financeira, a adentrarem, ou não, na situação de inadimplência, para posteriormente criar um 

<font color = "red">Meta de Acurácia Final:</font> ACC mínimo de 88%

<font color = "blue">Nesta etapa, realizarei o treinamento dos modelos de classificação, para posterior comparação e teste, a fim de verificar as métricas. 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importação dos dados 
X = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/X_pre_processado_5.csv")
y = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/y_pre_processado_5.csv")

In [6]:
X

,Renda_Anual,Anos_no_Emprego,Grau_Emprestimo,Valor,Taxa_Juros,Comprometimento_Renda,Inadimplencia_Historica,Tipo_Moradia_Hipoteca,Tipo_Moradia_Outro,Tipo_Moradia_Próprio,Motivo_Emprestimo_Educação,Motivo_Emprestimo_Empreendimento,Motivo_Emprestimo_Médico,Motivo_Emprestimo_Pessoal,Motivo_Emprestimo_Reformas Domésticas
0,-1.903478,0.196962,1,-1.551342,0.074325,-0.663416,0,0,0,1,1,0,0,0,0
1,-1.891412,-0.699305,0,-1.227123,-1.184422,1.077414,0,0,0,1,0,1,0,0,0
2,-1.887390,0.495717,3,-1.421655,1.207196,0.032916,0,0,0,1,0,1,0,0,0
3,-1.855214,1.093228,1,-1.389233,0.027122,0.032916,0,1,0,0,1,0,0,0,0
4,-1.847975,-1.296815,0,-1.443269,-1.137219,-0.199194,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34937,0.709549,-0.416085,4,2.555442,2.053350,1.309524,0,0,0,0,0,0,0,0,0
34938,-1.533943,-0.699305,3,-0.761657,1.393679,1.015354,1,0,0,0,0,0,0,1,0
34939,0.874144,0.959136,2,-0.120001,0.275724,-0.687167,1,1,0,0,0,0,0,0,0
34940,-1.112140,-0.998060,1,0.318936,-0.121854,1.975202,0,0,0,0,1,0,0,0,0


In [7]:
# Divisão em Dados de Treino e Teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X,
                                                        y,
                                                        test_size = 0.2,
                                                        random_state = 40)

# <font color = 'red'> --------------

# Regressão Logística 

In [8]:
modelo_v1_regressao = LogisticRegression()

In [9]:
modelo_v1_regressao.fit(X_treino, y_treino)

LogisticRegression()

In [10]:
previsoes_v1_LR = modelo_v1_regressao.predict(X_teste)

In [11]:
modelo_v1_metricas = {
 "Modelo": "Regressão Logística",
 "Versão": "1",
 "Detalhes": " - ",
 "Precision": round(precision_score(previsoes_v1_LR, y_teste), 3),
 "Recall": round(recall_score(previsoes_v1_LR, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v1_LR, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v1_LR, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v1_LR), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}

modelo_v1_metricas

{'Modelo': 'Regressão Logística',
 'Versão': '1',
 'Detalhes': ' - ',
 'Precision': 0.792,
 'Recall': 0.795,
 'F1_Score': 0.794,
 'Acurácia': 0.792,
 'AUC': 0.792}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Decision Tree com Grid Search</font>

In [12]:
# Parametros para grid
params_decision_tree = {
 "criterion": ["gini", "entropy"],
 "max_depth": [3, None],
 "min_samples_leaf": [1, 3, 10],
 "min_samples_split": [2, 3],
 "max_depth": [1,3,10]
}

In [13]:
clf_decision_tree = DecisionTreeClassifier()

In [14]:
# Grid Search
modelo_v2_decisionTree = GridSearchCV(clf_decision_tree,
                                      params_decision_tree,
                                      scoring='accuracy',
                                      cv = 5)

In [15]:
# Treinando 
modelo_v2_decisionTree.fit(X_treino, y_treino)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 3, 10],
                         'min_samples_leaf': [1, 3, 10],
                         'min_samples_split': [2, 3]},
             scoring='accuracy')

In [16]:
modelo_v2_decisionTree.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 3}

In [17]:
previsoes_v2_DT = modelo_v2_decisionTree.predict(X_teste)

In [18]:
modelo_v2_metricas = {
 "Modelo": "Decision Tree GS",
 "Versão": "2",
 "Detalhes": "gini",
 "Precision": round(precision_score(previsoes_v2_DT, y_teste), 3),
 "Recall": round(recall_score(previsoes_v2_DT, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v2_DT, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v2_DT, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v2_DT), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v2_metricas

{'Modelo': 'Decision Tree GS',
 'Versão': '2',
 'Detalhes': 'gini',
 'Precision': 0.706,
 'Recall': 0.972,
 'F1_Score': 0.817,
 'Acurácia': 0.841,
 'AUC': 0.842}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Random Forest sem Grid Search</font>

In [19]:
clf_random_forest = RandomForestClassifier(n_estimators = 100)

In [62]:
# Parametros
params_random_forest = {
    'max_depth': [None, 2, 10, 20],  # Profundidade máxima de cada árvore
    'min_samples_split': [2, 5, 10],  # Mínimo de amostras necessárias para dividir um nó
    'min_samples_leaf': [1, 2, 4],    # Mínimo de amostras necessárias em um nó folha
    'max_features': ['auto', 'sqrt'], # Número de características a serem consideradas para o melhor split
    'criterion': ['gini', 'entropy']
}

In [63]:
# Grid Search
modelo_v3_randomForest = GridSearchCV(clf_random_forest,
                                      params_random_forest,
                                      cv = 5)

In [20]:
clf_random_forest.fit(X_treino, y_treino)

RandomForestClassifier()

In [22]:
previsoes_v3_RF = clf_random_forest.predict(X_teste)

In [24]:
modelo_v3_metricas = {
 "Modelo": "Random Forest",
 "Versão": "3",
 "Detalhes": "Sem otimização",
 "Precision": round(precision_score(previsoes_v3_RF, y_teste), 3),
 "Recall": round(recall_score(previsoes_v3_RF, y_teste), 3),
 "F1_Score": round(f1_score(previsoes_v3_RF, y_teste), 3),
 "Acurácia": round(accuracy_score(previsoes_v3_RF, y_teste), 3),
 "AUC": round(roc_auc_score(y_teste, previsoes_v3_RF), 3) #AUC inverte a ordem dos dados -  primeiro real depois previsões
}
modelo_v3_metricas

{'Modelo': 'Random Forest',
 'Versão': '3',
 'Detalhes': 'Sem otimização',
 'Precision': 0.892,
 'Recall': 0.963,
 'F1_Score': 0.926,
 'Acurácia': 0.928,
 'AUC': 0.929}

# <font color = 'red'> --------------</font>

# <font color = 'red'>Random Forest com GS</font>